In [3]:
import pandas as pd
import math
import csv
import random
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (auc, classification_report, roc_auc_score, accuracy_score,
                             f1_score, log_loss, roc_curve, confusion_matrix, precision_score, recall_score)
from sklearn.preprocessing import StandardScaler
from math import sin, cos, sqrt, atan2, radians
import statsmodels.api as sm
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("D:/FORDHAM/March Madness/data/NCAA_Tourney_2002-2017.csv")
data.columns

Index(['Numot', 'Season', 'game_id', 'host_lat', 'host_long', 'host_site',
       'result', 'season', 'team1_adjde', 'team1_adjoe', 'team1_adjtempo',
       'team1_ap_final', 'team1_ap_preseason', 'team1_arate', 'team1_blockpct',
       'team1_coach_id', 'team1_coach_name', 'team1_coaches_before_final',
       'team1_coaches_preseason', 'team1_de', 'team1_f3grate', 'team1_fg2pct',
       'team1_fg3pct', 'team1_ftpct', 'team1_id', 'team1_lat', 'team1_long',
       'team1_oe', 'team1_opparate', 'team1_oppblockpct', 'team1_oppf3grate',
       'team1_oppfg2pct', 'team1_oppfg3pct', 'team1_oppftpct',
       'team1_oppstlrate', 'team1_pt_career_overall_losses',
       'team1_pt_career_school_losses', 'team1_pt_career_school_wins',
       'team1_pt_coach_season_losses', 'team1_pt_coach_season_wins',
       'team1_pt_overall_ff', 'team1_pt_overall_ncaa', 'team1_pt_overall_s16',
       'team1_pt_school_ff', 'team1_pt_school_ncaa', 'team1_pt_school_s16',
       'team1_pt_team_season_losses', 'tea

In [4]:
MM2 = data[['result', 'game_id','team1_adjde', 'team1_adjoe',
       'team1_adjtempo', 'team1_arate', 'team1_blockpct', 'team1_fg2pct',
       'team1_fg3pct', 'team1_ftpct', 'team1_id', 'team1_opparate',
       'team1_oppblockpct', 'team1_oppfg2pct', 'team1_oppfg3pct',
       'team1_oppftpct', 'team1_oppstlrate', 'team1_pt_career_overall_losses',
       'team1_pt_career_school_wins', 'team1_pt_overall_ncaa',
       'team1_pt_school_ncaa', 'team1_pt_team_season_losses',
       'team1_pt_team_season_wins', 'team1_rpi_rating',
       'team1_seed', 'team1_stlrate', 'team2_adjde', 'team2_adjoe',
       'team2_adjtempo', 'team2_arate', 'team2_blockpct', 'team2_f3grate',
       'team2_fg2pct', 'team2_fg3pct', 'team2_ftpct', 'team2_id',
       'team2_opparate', 'team2_oppblockpct', 'team2_oppf3grate',
       'team2_oppfg2pct', 'team2_oppfg3pct', 'team2_oppftpct',
       'team2_oppstlrate', 'team2_pt_career_overall_losses',
       'team2_pt_career_school_wins', 'team2_pt_overall_ncaa',
       'team2_pt_school_ncaa', 'team2_pt_team_season_losses',
       'team2_pt_team_season_wins', 'team2_rpi_rating',
       'team2_seed', 'team2_stlrate','host_lat', 'host_long', 'team1_lat', 'team1_long', 'team2_lat','team2_long',
           'team1_seed', 'team2_seed']]

In [5]:
elo2017=pd.read_csv("D:/FORDHAM/March Madness/data/2001ELO.csv")

team_elos = dict()
for index, row in elo2017.iterrows():
    team_elos[row['team_id']] = row['team_elo']

In [6]:
def get_elo(team_id):
    try:
        return team_elos[team_id]
    except:
        # base_elo
        team_elos[team_id] = 1500
        return team_elos[team_id]

def calc_elo(wteamid, lteamid):
    winner_rank = get_elo(wteamid)
    loser_rank = get_elo(lteamid)

    rank_diff = winner_rank - loser_rank
    exp = (rank_diff  * -1) / 400
    odds = 1 / (1 + math.pow(10, exp))
    # 
    if winner_rank < 2030:
        k = 32
    elif winner_rank >= 2030 and winner_rank < 2130:
        k = 24
    else:
        k = 16
    new_winner_rank = round(winner_rank + (k * (1 - odds)))
    new_rank_diff = new_winner_rank - winner_rank
    new_loser_rank = loser_rank - new_rank_diff

    return new_winner_rank, new_loser_rank

In [7]:
MMuse = data[['game_id','team1_id', 'team2_id', 'result', 'team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde',
             'host_lat', 'host_long', 'team1_lat', 'team1_long', 'team2_lat','team2_long', 'team1_pt_career_school_wins',
             'team1_seed', 'team2_seed']]

In [8]:
team1_nelo=list()
team2_nelo=list()

i=0
for index, row in MM2.iterrows():
    if row['result'] == 1: 
        wteam = row['team1_id']
        lteam = row['team2_id']
        new_team1_rank, new_team2_rank = calc_elo(wteam, lteam)
    else:
        wteam = row['team2_id']
        lteam = row['team1_id']
        new_team2_rank, new_team1_rank = calc_elo(wteam, lteam)
    i=i+1
    team_elos[row['team1_id']] = new_team1_rank
    team_elos[row['team2_id']] = new_team2_rank
    team1_nelo.append(new_team1_rank)
    team2_nelo.append(new_team2_rank)

In [9]:
data ={'game_id':MM2['game_id'], 'team1_nelo': team1_nelo, 'team2_nelo':team2_nelo, 'team1_id':MM2['team1_id'],
      'team2_id':MM2['team2_id'], 'result':MM2['result'], 'team1_pt_career_school_wins':MM2['team1_pt_career_school_wins'],
       'team1_adjoe':MM2['team1_adjoe'],'team2_adjoe':MM2['team2_adjoe'], 'team1_adjde':MM2['team1_adjde'], 
       'team2_adjde':MM2['team2_adjde'],'host_lat':MM2['host_lat'], 'host_long':MM2['host_long'], 'team1_lat':MMuse['team1_lat'], 
       'team1_long':MMuse['team1_long'], 'team2_lat':MMuse['team2_lat'],'team2_long':MMuse['team2_long'], 'team1_seed':MMuse['team1_seed'],
       'team2_seed':MMuse['team2_seed']}
type (data)
MMuse = pd.DataFrame(data)

In [10]:
# calcylate difference
from math import sin, cos, sqrt, atan2, radians
# define the function distance
def distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance


In [11]:
# read the data
MMuse['dist1'] = MMuse.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team1_lat'], row['team1_long']), axis=1)
MMuse['dist2'] = MMuse.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team2_lat'], row['team2_long']), axis=1)

MMuse['diff_dist'] = MMuse['dist1'] - MMuse['dist2']

In [12]:
MMuse.columns

Index(['game_id', 'host_lat', 'host_long', 'result', 'team1_adjde',
       'team1_adjoe', 'team1_id', 'team1_lat', 'team1_long', 'team1_nelo',
       'team1_pt_career_school_wins', 'team1_seed', 'team2_adjde',
       'team2_adjoe', 'team2_id', 'team2_lat', 'team2_long', 'team2_nelo',
       'team2_seed', 'dist1', 'dist2', 'diff_dist'],
      dtype='object')

In [19]:
MMlive_elo = MMuse[['game_id', 'result', 'team1_adjde',
       'team1_adjoe', 'team1_id', 'team1_nelo', 'team1_pt_career_school_wins', 'team1_seed', 'team2_adjde',
       'team2_adjoe', 'team2_id','team2_nelo','team2_seed', 'dist1', 'dist2', 'diff_dist']]
MMlive_elo.to_csv("D:/FORDHAM/March Madness/data/MMlive_elo2.csv")

In [13]:
data = pd.read_csv("D:/FORDHAM/March Madness/data/MMlive_elo2002_2017.csv")
data.columns

Index(['team2_nnelo', 'team1_nnelo', 'game_id', 'result', 'team1_adjde',
       'team1_adjoe', 'team1_id', 'team1_nelo', 'team1_pt_career_school_wins',
       'team1_seed', 'team2_adjde', 'team2_adjoe', 'team2_id', 'team2_nelo',
       'team2_seed', 'dist1', 'dist2', 'diff_dist'],
      dtype='object')

In [14]:
MMuse = data[['team1_nnelo', 'team2_nnelo', 'team1_id', 'team1_nelo',
       'team1_pt_career_school_wins', 'team1_seed', 'team2_adjde',
       'team2_adjoe', 'team2_id', 'team2_nelo', 'team2_seed', 'diff_dist',
       'game_id', 'result', 'team1_adjde', 'team1_adjoe']]

# Logistic Model with 6 variables+diff_dist

In [15]:
scaler = StandardScaler()
mm_train_scaled = scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde','diff_dist']])
logit = LogisticRegression()
smlogit = logit.fit(mm_train_scaled, MMuse['result'])

In [16]:
smlogit = sm.Logit(MMuse['result'], MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde','diff_dist']]).fit()
smlogit.summary()

Optimization terminated successfully.
         Current function value: 0.524832
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 result   No. Observations:                 1041
Model:                          Logit   Df Residuals:                     1034
Method:                           MLE   Df Model:                            6
Date:                Sat, 17 Mar 2018   Pseudo R-squ.:                  0.2418
Time:                        20:53:53   Log-Likelihood:                -546.35
converged:                       True   LL-Null:                       -720.59
                                        LLR p-value:                 3.262e-72
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
team1_nnelo     0.0005      0.000      0.942      0.346      -0.000       0.001
team2_nnelo    -0.0016      0.000     -3.325      0.001      -0.003      -0.001
team1_adjoe     0.1315      0.017      7.758      0.000       0.098       0.165
team2_adjoe    -0.0854      0.016     -5.234      0.000      -0.117      -0.053
team1_adjde    -0.1391      0.018     -7.923      0.000      -0.173      -0.105
team2_adjde     0.1061      0.017      6.166      0.000       0.072       0.140
diff_dist      -0.0001   6.37e-05     -1.644      0.100      -0.000    2.01e-05
===============================================================================
"""

In [18]:
MMuse['preds2'] = logit.predict_proba(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde','diff_dist']]))[:,1]
MMuse['prediction'] = logit.predict(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))

In [19]:
accuracy_score(MMuse['result'], MMuse['prediction'],)

0.72526416906820368

In [20]:
log_loss(MMuse['result'].values, MMuse['preds2'].values)

0.52471766782592799

In [21]:
scores = cross_val_score(logit, mm_train_scaled, MMuse['result'], cv=5)
scores   
scores.mean()

0.7205510482398354

# Logistic Model with 6 variables + team1_pt_career_school_wins

In [17]:
scaler = StandardScaler()
mm_train_scaled = scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde','team1_pt_career_school_wins']])
logit = LogisticRegression()
logit.fit(mm_train_scaled, MMuse['result'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
smlogit = sm.Logit(MMuse['result'], MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde','team1_pt_career_school_wins']]).fit()
smlogit.summary()

Optimization terminated successfully.
         Current function value: 0.525887
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 result   No. Observations:                 1041
Model:                          Logit   Df Residuals:                     1034
Method:                           MLE   Df Model:                            6
Date:                Mon, 12 Mar 2018   Pseudo R-squ.:                  0.2403
Time:                        15:29:22   Log-Likelihood:                -547.45
converged:                       True   LL-Null:                       -720.59
                                        LLR p-value:                 9.655e-72
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
team1_nnelo                     0.0004      0.001      0.733      0.464      -0.001       0.001
team2_nnelo                    -0.0016      0.000     -3.380      0.001      -0.003      -0.001
team1_adjoe                     0.1317      0.017      7.732      0.000       0.098       0.165
team2_adjoe                    -0.0852      0.016     -5.240      0.000      -0.117      -0.053
team1_adjde                    -0.1406      0.018     -7.952      0.000      -0.175      -0.106
team2_adjde                     0.1086      0.017      6.329      0.000       0.075       0.142
team1_pt_career_school_wins     0.0003      0.000      0.722      0.471      -0.000       0.001
===============================================================================================
"""

In [19]:
MMuse['preds'] = logit.predict_proba(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde','team1_pt_career_school_wins']]))[:,1]
MMuse['prediction'] = logit.predict(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'team1_pt_career_school_wins']]))

In [20]:
accuracy_score(MMuse['result'], MMuse['prediction'],)

0.72334293948126804

In [21]:
log_loss(MMuse['result'].values, MMuse['preds'].values)

0.52581601828731328

# Logistic Model with 6 variables

In [71]:
scaler = StandardScaler()
mm_train_scaled = scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']])
logit = LogisticRegression()
logit.fit(mm_train_scaled, MMuse['result'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [72]:
smlogit = sm.Logit(MMuse['result'], MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde','team1_pt_career_school_wins']]).fit()
smlogit.summary()

Optimization terminated successfully.
         Current function value: 0.525887
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 result   No. Observations:                 1041
Model:                          Logit   Df Residuals:                     1034
Method:                           MLE   Df Model:                            6
Date:                Fri, 16 Mar 2018   Pseudo R-squ.:                  0.2403
Time:                        15:25:20   Log-Likelihood:                -547.45
converged:                       True   LL-Null:                       -720.59
                                        LLR p-value:                 9.655e-72
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
team1_nnelo                     0.0004      0.001      0.733      0.464      -0.001       0.001
team2_nnelo                    -0.0016      0.000     -3.380      0.001      -0.003      -0.001
team1_adjoe                     0.1317      0.017      7.732      0.000       0.098       0.165
team2_adjoe                    -0.0852      0.016     -5.240      0.000      -0.117      -0.053
team1_adjde                    -0.1406      0.018     -7.952      0.000      -0.175      -0.106
team2_adjde                     0.1086      0.017      6.329      0.000       0.075       0.142
team1_pt_career_school_wins     0.0003      0.000      0.722      0.471      -0.000       0.001
===============================================================================================
"""

In [73]:
MMuse['preds2'] = logit.predict_proba(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde']]))[:,1]
MMuse['prediction'] = logit.predict(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde']]))

In [74]:
accuracy_score(MMuse['result'], MMuse['prediction'],)

0.72622478386167144

In [75]:
log_loss(MMuse['result'].values, MMuse['preds2'].values)

0.52604379561686443

In [380]:
MMuse.to_csv('D:/FORDHAM/March Madness/data/output2012-2017.csv')

#  NN with 6 variables + diff_dist

In [22]:
MMuse['result_str'] = MMuse['result'].astype(str)
from sklearn.neural_network import MLPClassifier

In [23]:
scaler = StandardScaler()
mm_train_scaled = scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde', 'diff_dist']])
NN = MLPClassifier()
NN.fit(mm_train_scaled, MMuse['result_str'])         

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [24]:
MMuse['preds3'] = NN.predict_proba(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))[:,1]
MMuse['prediction'] = NN.predict(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))

In [25]:
accuracy_score(MMuse['result_str'], MMuse['prediction'],)

0.74063400576368876

In [26]:
log_loss(MMuse['result_str'].values, MMuse['preds3'].values)

0.50634364649340835

In [67]:
MMuse.to_csv('D:/FORDHAM/March Madness/data/2002-2017_result.csv')

In [177]:
len(data)

2278

In [27]:
data = pd.read_csv("D:/FORDHAM/March Madness/data/NCAA_Tourney_2018.csv")
data.columns

Index(['game_id', 'team1_id', 'team2_id', 'season', 'team1_seed', 'team2_seed',
       'team1_position', 'team2_position', 'strongseed', 'weakseed',
       'team1_region', 'team2_region', 'slot', 'host', 'host_lat', 'host_long',
       'team1_name', 'team2_name', 'team1_rpi_rating', 'team2_rpi_rating',
       'team1_fg2pct', 'team1_fg3pct', 'team1_ftpct', 'team1_blockpct',
       'team1_oppfg2pct', 'team1_oppfg3pct', 'team1_oppftpct',
       'team1_oppblockpct', 'team1_fg3rate', 'team1_oppfg3rate', 'team1_arate',
       'team1_opparate', 'team1_stlrate', 'team1_oppstlrate', 'team2_fg2pct',
       'team2_fg3pct', 'team2_ftpct', 'team2_blockpct', 'team2_oppfg2pct',
       'team2_oppfg3pct', 'team2_oppftpct', 'team2_oppblockpct',
       'team2_fg3rate', 'team2_oppfg3rate', 'team2_arate', 'team2_opparate',
       'team2_stlrate', 'team2_oppstlrate', 'team1_tempo', 'team1_adjtempo',
       'team1_oe', 'team1_adjoe', 'team1_de', 'team1_adjde', 'team2_tempo',
       'team2_adjtempo', 'team2_o

In [189]:
len(data)

2278

In [28]:
elo2017=pd.read_csv("D:/FORDHAM/March Madness/data/2017ELO.csv")
team_elos = dict()
for index, row in elo2017.iterrows():
    team_elos[row['team_id']] = row['team_elo']

In [195]:
a=1437
team_elos[a]

2142

In [29]:
team1_nelo=list()
team2_nelo=list()
i=0
for index, row in data.iterrows():
    i=i+1
    try: 
        team1_nelo.append(team_elos[row['team1_id']])
        team2_nelo.append(team_elos[row['team2_id']])
    except:
        team1_nelo.append(1500)
        team2_nelo.append(1500)
print (i, len(team1_nelo), len(team2_nelo))

2278 2278 2278


In [30]:
MM2018 ={'game_id':data['game_id'], 'team1_nelo': team1_nelo, 'team2_nelo':team2_nelo, 'team1_id':data['team1_id'],
      'team2_id':data['team2_id'], 'team1_adjoe':data['team1_adjoe'],'team2_adjoe':data['team2_adjoe'], 'team1_adjde':data['team1_adjde'], 
       'team2_adjde':data['team2_adjde'],'host_lat':data['host_lat'], 'host_long':data['host_long'], 'team1_lat':data['team1_lat'], 
       'team1_long':data['team1_long'], 'team2_lat':data['team2_lat'],'team2_long':data['team2_long']}

In [31]:
MM2018 = pd.DataFrame(MM2018)

In [32]:
MM2018['dist1'] = MM2018.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team1_lat'], row['team1_long']), axis=1)
MM2018['dist2'] = MM2018.apply(lambda row: distance(row['host_lat'], row['host_long'], row['team2_lat'], row['team2_long']), axis=1)

MM2018['diff_dist'] = MM2018['dist1'] - MM2018['dist2']

In [33]:
MM2018['team1_name'] = data['team1_name']
MM2018['team2_name'] = data['team2_name']

In [34]:
MM2018.head()

,game_id,host_lat,host_long,team1_adjde,team1_adjoe,team1_id,team1_lat,team1_long,team1_nelo,team2_adjde,team2_adjoe,team2_id,team2_lat,team2_long,team2_nelo,dist1,dist2,diff_dist,team1_name,team2_name
0,2018-1437-1345,42.3601,-71.0589,99.9,122.8,1437,40.039388,-75.345457,2142,96.7,118.5,1345,40.428206,-86.914435,1932,441.868828,1338.323024,-896.454196,Villanova,Purdue
1,2018-1437-1403,42.3601,-71.0589,99.9,122.8,1437,40.039388,-75.345457,2142,94.2,109.2,1403,33.580309,-101.876917,1888,441.868828,2854.475981,-2412.607153,Villanova,Texas Tech
2,2018-1437-1455,42.3601,-71.0589,99.9,122.8,1437,40.039388,-75.345457,2142,101.9,117.8,1455,37.716484,-97.296733,1972,441.868828,2283.488269,-1841.619441,Villanova,Wichita St
3,2018-1437-1452,42.3601,-71.0589,99.9,122.8,1437,40.039388,-75.345457,2142,98.5,113.0,1452,39.634680,-79.953926,1966,441.868828,805.408890,-363.540062,Villanova,West Virginia
4,2018-1437-1196,42.3601,-71.0589,99.9,122.8,1437,40.039388,-75.345457,2142,100.6,109.7,1196,29.638248,-82.361195,1946,441.868828,1739.142669,-1297.273841,Villanova,Florida


In [35]:
MM2018['preds'] = NN.predict_proba(scaler.fit_transform(MM2018[['team1_nelo','team2_nelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))[:,1]
MM2018['prediction'] = NN.predict(scaler.fit_transform(MM2018[['team1_nelo','team2_nelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))

In [204]:
MM2018.to_csv('D:/FORDHAM/March Madness/data/Berserker_resultv2.csv')

In [245]:
#MMuse.to_csv('D:/FORDHAM/March Madness/data/roc.csv')
MM2018.columns

Index(['game_id', 'host_lat', 'host_long', 'team1_adjde', 'team1_adjoe',
       'team1_id', 'team1_lat', 'team1_long', 'team1_nelo', 'team2_adjde',
       'team2_adjoe', 'team2_id', 'team2_lat', 'team2_long', 'team2_nelo',
       'dist1', 'dist2', 'diff_dist', 'team1_name', 'team2_name', 'preds',
       'prediction'],
      dtype='object')

In [37]:
round1id=pd.read_csv("D:/FORDHAM/March Madness/data/round1id.csv")

In [39]:
round1result = pd.merge(MM2018, round1id, how='inner', on ='game_id')#, on= 'game_id')
round1result.head()

,game_id,host_lat,host_long,team1_adjde,team1_adjoe,team1_id,team1_lat,team1_long,team1_nelo,team2_adjde,...,team2_lat,team2_long,team2_nelo,dist1,dist2,diff_dist,team1_name,team2_name,preds,prediction
0,2018-1437-1347,40.4417,-80.0000,99.9,122.8,1437,40.039388,-75.345457,2142,100.5,...,37.136914,-80.549731,1932,397.676825,370.665213,27.011612,Villanova,Radford,0.963394,1
1,2018-1345-1168,42.3314,-83.0458,96.7,118.5,1345,40.428206,-86.914435,1932,101.5,...,33.883627,-117.885349,1887,386.024594,3165.668423,-2779.643830,Purdue,CS Fullerton,0.942205,1
2,2018-1403-1372,32.7767,-96.7970,94.2,109.2,1403,33.580309,-101.876917,1888,94.0,...,31.585620,-94.585460,1864,481.238753,246.765406,234.473347,Texas Tech,SF Austin,0.570748,1
3,2018-1455-1267,32.7150,-117.1625,101.9,117.8,1455,37.716484,-97.296733,1972,102.9,...,38.422610,-82.430650,1830,1884.774445,3186.314819,-1301.540375,Wichita St,Marshall,0.843927,1
4,2018-1452-1293,32.7150,-117.1625,98.5,113.0,1452,39.634680,-79.953926,1966,97.5,...,36.612410,-88.323493,1816,3402.698956,2663.507524,739.191432,West Virginia,Murray St,0.394118,0


In [40]:
team1_r1nelo=list()
team2_r1nelo=list()

i=0
for index, row in round1result.iterrows():
    if row['prediction'] == 1: 
        wteam = row['team1_id']
        lteam = row['team2_id']
        new_team1_rank, new_team2_rank = calc_elo(wteam, lteam)
    else:
        wteam = row['team2_id']
        lteam = row['team1_id']
        new_team2_rank, new_team1_rank = calc_elo(wteam, lteam)
    i=i+1
    team_elos[row['team1_id']] = new_team1_rank
    team_elos[row['team2_id']] = new_team2_rank
    #row['team1_r1elo'] = new_team1_rank
    #row['team2_r1elo'] = new_team2_rank
    team1_r1nelo.append(new_team1_rank)
    team2_r1nelo.append(new_team2_rank)

In [41]:
round1result['team1_r1elo'] = team1_r1nelo
round1result['team2_r1elo'] = team2_r1nelo

In [42]:
round1result.to_csv('D:/FORDHAM/March Madness/data/R1ELO.csv')

In [27]:
scores = cross_val_score(NN, mm_train_scaled, MMuse['result_str'], cv=5)
scores   
scores.mean()

0.7224649238379216

In [33]:
from sklearn.metrics import roc_curve, auc

In [30]:
fpr = dict()
tpr = dict()
roc_auc = dict()

In [34]:
fpr, tpr, threshold = metrics.roc_curve(MMuse['result_str'], MMuse['preds'])
roc_auc = metrics.auc(fpr, tpr)

NameError: name 'metrics' is not defined

In [ ]:
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

# KNN with 6 variables + diff_dist

In [90]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(mm_train_scaled, MMuse['result_str']) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [91]:
MMuse['preds4'] = neigh.predict_proba(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))[:,1]
MMuse['prediction'] = neigh.predict(scaler.fit_transform(MMuse[['team1_nnelo','team2_nnelo', 'team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde', 'diff_dist']]))

In [93]:
accuracy_score(MMuse['result_str'], MMuse['prediction'],)

0.77905859750240158

In [92]:
log_loss(MMuse['result_str'].values, MMuse['preds4'].values)

0.43004718278424919

In [94]:
scores = cross_val_score(neigh, mm_train_scaled, MMuse['result_str'], cv=5)
scores   
scores.mean()

0.68392853015278643

# Logistic Model with 4 variables

In [86]:
mm2_train_scaled = scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']])
logit.fit(mm2_train_scaled, MMuse['result'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [87]:
smlogit = sm.Logit(MMuse['result'], MMuse[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]).fit()
smlogit.summary()

Optimization terminated successfully.
         Current function value: 0.532072
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 result   No. Observations:                 1041
Model:                          Logit   Df Residuals:                     1037
Method:                           MLE   Df Model:                            3
Date:                Fri, 16 Mar 2018   Pseudo R-squ.:                  0.2313
Time:                        15:38:21   Log-Likelihood:                -553.89
converged:                       True   LL-Null:                       -720.59
                                        LLR p-value:                 5.823e-72
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
team1_adjoe     0.1396      0.013     10.365      0.000       0.113       0.166
team2_adjoe    -0.1186      0.013     -9.114      0.000      -0.144      -0.093
team1_adjde    -0.1572      0.016     -9.749      0.000      -0.189      -0.126
team2_adjde     0.1328      0.016      8.462      0.000       0.102       0.164
===============================================================================
"""

In [88]:
MMuse['preds1'] = logit.predict_proba(scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde']]))[:,1]
MMuse['prediction'] = logit.predict(scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe',
                                                                'team1_adjde', 'team2_adjde']]))

In [536]:
accuracy_score(MMuse['result'], MMuse['prediction'],)

0.71757925072046114

In [89]:
log_loss(MMuse['result'].values, MMuse['preds1'].values)

0.53136304708782878

In [538]:
scores = cross_val_score(logit, mm_train_scaled, MMuse['result_str'], cv=5)
scores   
scores.mean()

0.7205510482398354

# NN Model with 4 variables

In [82]:

NN2 = MLPClassifier()
NN2.fit(mm2_train_scaled, MMuse['result_str'])  

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [83]:
MMuse['preds'] = NN2.predict_proba(scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))[:,1]
MMuse['prediction'] = NN2.predict(scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))

In [84]:
accuracy_score(MMuse['result_str'], MMuse['prediction'],)

0.72046109510086453

In [85]:
log_loss(MMuse['result_str'].values, MMuse['preds'].values)

0.52592552003061666

# draft for validataion use

In [145]:
from sklearn.neural_network import MLPClassifier

In [155]:
NN = MLPClassifier()
NN.fit(mm2_train_scaled, MMuse['result_str'])                         

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [156]:
MMuse['preds'] = NN.predict_proba(scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))[:,1]
MMuse['prediction'] = NN.predict(scaler.fit_transform(MMuse[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))

In [158]:
accuracy_score(MMuse['result_str'], MMuse['prediction'],)

0.72622478386167144

In [159]:
log_loss(MMuse['result'].values, MMuse['preds'].values)

0.52244322596315729

In [74]:
data = pd.read_csv("D:/FORDHAM/March Madness/data/NCAA_Tourney_2017.csv")

In [75]:
MM2017 = data[['result', 'team1_adjde', 'team1_adjoe',
       'team1_adjtempo', 'team1_arate', 'team1_blockpct', 'team1_fg2pct',
       'team1_fg3pct', 'team1_ftpct', 'team1_id', 'team1_opparate',
       'team1_oppblockpct', 'team1_oppfg2pct', 'team1_oppfg3pct',
       'team1_oppftpct', 'team1_oppstlrate', 'team1_pt_career_overall_losses',
       'team1_pt_career_school_wins', 'team1_pt_overall_ncaa',
       'team1_pt_school_ncaa', 'team1_pt_team_season_losses',
       'team1_pt_team_season_wins', 'team1_rpi_rating',
       'team1_seed', 'team1_stlrate', 'team2_adjde', 'team2_adjoe',
       'team2_adjtempo', 'team2_arate', 'team2_blockpct', 'team2_f3grate',
       'team2_fg2pct', 'team2_fg3pct', 'team2_ftpct', 'team2_id',
       'team2_opparate', 'team2_oppblockpct', 'team2_oppf3grate',
       'team2_oppfg2pct', 'team2_oppfg3pct', 'team2_oppftpct',
       'team2_oppstlrate', 'team2_pt_career_overall_losses',
       'team2_pt_career_school_wins', 'team2_pt_overall_ncaa',
       'team2_pt_school_ncaa', 'team2_pt_team_season_losses',
       'team2_pt_team_season_wins', 'team2_rpi_rating',
       'team2_seed', 'team2_stlrate']]

In [83]:
i=0
team1_nelo=list()
team2_nelo=list()
for index, row in MM2017.iterrows():
    if row['result'] == 1: 
        wteam = row['team1_id']
        lteam = row['team2_id']
        new_team1_rank, new_team2_rank = calc_elo(wteam, lteam)
    else:
        wteam = row['team2_id']
        lteam = row['team1_id']
        new_team2_rank, new_team1_rank = calc_elo(wteam, lteam)
    i=i+1
    team_elos[row['team1_id']] = new_team1_rank
    team_elos[row['team2_id']] = new_team2_rank
    team1_nelo.append(new_team1_rank)
    team2_nelo.append(new_team2_rank)

In [84]:
data2 = {'team1_nelo': team1_nelo, 'team2_nelo':team2_nelo, 'team1_id':MM2017['team1_id'],
      'team2_id':MM2017['team2_id'], 'result':MM2017['result'],'team1_adjoe':MM2017['team1_adjoe'],
         'team2_adjoe':MM2017['team2_adjoe'], 'team1_adjde':MM2017['team1_adjde'], 'team2_adjde':MM2017['team2_adjde']}
MM2017use = pd.DataFrame(data2)

In [78]:
#mm2_train_scaled = scaler.fit_transform(MM2017use[['team1_nelo','team2_nelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']])
#RanFor= RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
#RanFor.fit(mm2_train_scaled, MM2017use['result'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [90]:
#MM2017use['preds'] = RanFor.predict_proba(scaler.fit_transform(MM2017use[['team1_nelo','team2_nelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))[:,1]
MM2017use['prediction'] = RanFor.predict(scaler.fit_transform(MM2017use[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))

In [91]:
accuracy_score(MM2017use['result'], MM2017use['prediction'],)

0.46031746031746029

In [46]:
MM2017use['preds'] = logit.predict_proba(scaler.fit_transform(MM2017use[['team1_nelo','team2_nelo','team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))[:,1]
MM2017use['prediction'] = logit.predict(scaler.fit_transform(MM2017use[['team1_nelo','team2_nelo', 'team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))

ValueError: X has 6 features per sample; expecting 4

In [37]:
accuracy_score(MM2017use['result'], MM2017use['prediction'],)

NameError: name 'MM2017use' is not defined

In [68]:
MM2017use['preds'] = logit.predict_proba(scaler.fit_transform(MM2017use[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))[:,1]
MM2017use['prediction'] = logit.predict(scaler.fit_transform(MM2017use[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))

In [69]:
accuracy_score(MM2017use['result'], MM2017use['prediction'],)

0.46031746031746029

In [66]:
MM2017use['preds'] = RanFor.predict_proba(scaler.fit_transform(MM2017use[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))[:,1]
MM2017use['prediction'] = RanFor.predict(scaler.fit_transform(MM2017use[['team1_adjoe','team2_adjoe', 'team1_adjde', 'team2_adjde']]))

In [195]:
log_loss(MMuse['result'].values, MMuse['preds'].values)

0.50612390073648161

In [211]:
accuracy_score(MM2017use['result'], MM2017use['prediction'],)

NameError: name 'MM2017use' is not defined

# _______________________marked___________for distance